# Exploration des données

## 1. Préparation

In [1]:
import os
from PIL import Image, UnidentifiedImageError
from PIL.TiffTags import TAGS
import pandas as pd
import numpy as np
import cv2

import pytesseract
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe" #il faut mettre là où tesseract est installé

import time
from tqdm.notebook import tqdm

In [2]:
#################################### A MODIFIER SELON SA PROPRE ARBORESCENCE
project_path = 'D:/Projet/mai25_bds_extraction/' 
####################################

raw_data_path = os.path.join(project_path, 'data', 'raw')
processed_data_path = os.path.join(project_path, 'data', 'processed')

rvl_cdip_images_path = os.path.join(raw_data_path, 'RVL-CDIP', 'images')

## 2. Fichiers images RVL-CDIP

In [3]:
def get_rvl_cdip_image_files(path):
    tmp_list = []
    for foldername, _, filenames in os.walk(path):
        if filenames:
            filename = filenames[0]
            # we check that the structure is relevant to our expectation with 2 assert
            assert len(filenames) == 1
            if filename.startswith('.'): # avoid to consider files like .DS_Store on mac
                continue
            assert filename.endswith(".tif"), f"{foldername},{filename}"
            tmp_list.append([
                os.path.relpath(foldername, rvl_cdip_images_path),
                filename
            ])
    return pd.DataFrame(tmp_list, columns = ["relative_path", "filename"])


In [4]:
t = time.time()
df = get_rvl_cdip_image_files(rvl_cdip_images_path)
print(f"Duree d'exécution: {time.time() - t:.3f} secondes.")
print(f"({len(df)} images traitées)\n")

df.head()

Duree d'exécution: 55.055 secondes.
(400000 images traitées)



,relative_path,filename
0,imagesa\a\a\a\aaa06d00,50486482-6482.tif
1,imagesa\a\a\a\aaa08d00,2072197187.tif
2,imagesa\a\a\a\aaa09e00,2029372116.tif
3,imagesa\a\a\a\aaa10c00,2085133627a.tif
4,imagesa\a\a\a\aaa11d00,515558347+-8348.tif


## Que veut-on extraire des images ? 
- format (en fait tout est en .tiff)
- taille (longueur et largeur, normalement, la plus grande dimension de l'image devrait être mise à 1000)
- mode (ici en fait ce sont toutes des images en nuances de gris, sur 8 bits)
- nombre de pages par image
- niveau de flou - grace à la variance du laplacien
- le bruit de l'image (quand des points apparaîssent sur un fond censé être blanc par exemple)
- densité (quantité de pixels noirs par rapport à la quantité de pixels blancs)

Lors de la 2e itération, je rajoute:
- le nombre de blocs de texte
- la quantité de blanc autour du contenu/présence d'une bordure
- la présence de motifs réguliers
- la détection des caractères de texte

In [5]:
# on écrit une fonction pour récupérer le nombre de pages par fichier
def nombre_pages(img):
    try:
        count = 0
        while True:
            try:
                img.seek(count)
                count += 1
            except EOFError:
                break
        return count
    except UnidentifiedImageError:
        return None

In [6]:
# on écrit une fonction pour évaluer le "flou"
def flou(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        return None
    fl = cv2.Laplacian(image, cv2.CV_64F).var()/(image.shape[0] * image.shape[1])
    #on normalise la variance du laplacien par la taille de l'image
    return fl
    # Plus la variance est basse, plus l'image est floue
    # attention, la variance du laplacien peut être artificiellement élevée si l'image a beaucoup de grain

In [7]:
# on écrit une fonction pour évaluer le grain de l'image
# plus le grain est élévé, moins l'image est lisible

def bruit(image_path):
    """ l'idée derrière cette fonction:
    - l'image est lissée par une gaussienne, les grains disparaîssent
    - on compare l'image lissée à l'image initiale
    - la différence c'est du bruit """
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        return None
    blurred = cv2.GaussianBlur(image, (5, 5), 0)
    residual = cv2.absdiff(image, blurred)
    return np.mean(residual)  # Intensité moyenne du bruit

In [8]:
#on écrit une fonction pour connaître la proportion de pixels blancs et la proportion de pixels noirs
def ratio_blanc_noir(image_path, threshold=200):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        return None, None
    white_ratio = np.sum(img > threshold) / img.size
    black_ratio = np.sum(img < 50) / img.size
    return white_ratio, black_ratio

### On teste sur la première image

In [15]:
# Données disponibles sur les images:
relative_filepath, filename = df.iloc[1, :]
file_path = os.path.join(rvl_cdip_images_path, relative_filepath, filename)
print("Flou:", flou(file_path))
print("Bruit:", bruit(file_path))
ratio_b, ratio_n = ratio_blanc_noir(file_path, threshold=200)
print("ratio_blanc : ", ratio_b)
print("ratio_noir : ", ratio_n)
with Image.open(file_path) as img:
    print("Format:", img.format)
    print("Size:", img.size)
    print("Mode:", img.mode)
    print("Nb_pages:", nombre_pages(img))

    # Attributes and methods
    #print("\nAttributes and methods list:")
    #print(dir(img))

    # Attributes
    #print("\nAttributes:")
    #for key, value in img.__dict__.items():
    #    print(f"{key}: {value}")

    # Metadata tags
    #print("\nTIFF Metadata:")
    #for tag, value in img.tag.items():
    #    tag_name = TAGS.get(tag, tag)
    #    print(f"{tag_name}: {value}")


# VOIR ICI pour tout ce que l'on peut extraire: https://pillow.readthedocs.io/en/stable/reference/Image.html#PIL.Image.Image

Flou: 0.01744088797500862
Bruit: 14.017836870026525
Features: {'num_lines': 121, 'num_words': 408, 'digit_ratio': 0.02743305011474167}
Blocs: 4
Bordire: {'top_margin': 0.059, 'bottom_margin': 0.022, 'left_margin': 0.027851458885941646, 'right_margin': 0.04111405835543767}
Motif: 0.9805117161056777
ratio_blanc :  0.8546657824933687
ratio_noir :  0.05696551724137931
Format: TIFF
Size: (754, 1000)
Mode: L
Nb_pages: 1


In [16]:
# A réécrire: 1 seule fonction et 1 apply
def get_image_data(file_path): # [TODO: liste des parametres à affiner]
    if not os.path.exists(file_path):
        print(f"[Manquant] {file_path}")
        return [np.nan] * 9  # 9 colonnes
    try:
        nettete = flou(file_path)
        bruit_grain = bruit(file_path)
        ratio_b, ratio_n = ratio_blanc_noir(file_path, threshold=200)
        bloc = blocs(file_path)
        feature = features(file_path)
        bord = bordure(file_path)
        motif = motifs(file_path)
        with Image.open(file_path) as img:
            format_ = img.format
            width, height = img.size 
            mode = img.mode
            nb_pages = nombre_pages(img)
        return format_, width, height, mode, nb_pages, nettete, bruit_grain, ratio_b, ratio_n, bloc, feature, bord, motif
    except Exception as e:
        print(f"[Erreur lecture] {file_path} → {e}")
        return [np.nan] * 9

def get_image_data_to_df(df):
    tqdm.pandas()  # active le support de tqdm pour pandas, pour faire une belle barre d'avancement
    def f(row):
        relative_filepath, filename = df.iloc[0, :]
        file_path = os.path.join(rvl_cdip_images_path, relative_filepath, filename)
        relative_filepath, filename = row["relative_path"], row["filename"]
        file_path = os.path.join(rvl_cdip_images_path, relative_filepath, filename)
        return get_image_data(file_path)
    return pd.DataFrame(
        df.apply(f, axis = 1).tolist(),
        columns = ["format", "width", "height", "mode", "nb_pages", "nettete", "bruit_grain", "ratio_b", "ratio_n", "bloc", "feature", "bord", "motif"],
        index = df.index
    )

In [17]:
t = time.time()
df = pd.concat([df, get_image_data_to_df(df)], axis = 1)
print(f"Duree d'exécution: {time.time() - t:.3f} secondes.")
print(f"({len(df)} images traitées)\n")
df

KeyboardInterrupt: 

In [18]:
#df.to_pickle("rvl_cdip_draft0.pkl")
df.to_csv(os.path.join(project_path, 'data', 'extracted', 'rvl_cdip_draft3.csv'))